In [1]:
from plot import *
from fft import *
from maths import rmse, mae, fit_kochanek_bartels_spline, hermite_interpolate


In [28]:
def fit_dft2_hermite_interpolated_at_time(filename: str, variable: str, time: int, quantile: float, skip_levels: int):
    data = load_variable_at_time(filename, variable, time)
    prediction = np.zeros((36, 361, 576), dtype="float32")

    preds = []
    levels = sorted(set(range(0, 36, skip_levels)).union({35}))
    for lev in levels:
        fft = dft2_at_time_and_level(data[lev], quantile=quantile)
        preds.append(idft2_at_time_and_level(*fft))

    tensions = []
    biases = []

    for lev in tqdm(range(36)):
        if lev % skip_levels == 0:
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])

        best_tension, best_bias = fit_kochanek_bartels_spline(data[lev], preds, pred_idx, t)
        tensions.append(best_tension)
        biases.append(best_bias)

    for lev in range(36):
        if lev in levels:
            prediction[lev] = preds[levels.index(lev)]
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])
        prediction[lev] = hermite_interpolate(preds, pred_idx, t, tensions[lev - pred_idx - 1], 0)

    data_float32 = data.astype("float32")

    print(f"Original Stdev: {data_float32.std()} m/s")
    print(f"Predicted RMSE: {rmse(data_float32, prediction)} m/s")
    print(f"Predicted MAE:  {mae(data_float32, prediction)} m/s")
    print(f"Size/level: {sum(el.nbytes for el in fft) / (1000 ** 2)} MB")
    print(f"Size/time: {sum(el.nbytes for el in fft) * len(levels) / (1000 ** 2)} MB")
    print(f"Size/day: {sum(el.nbytes for el in fft) * len(levels) * 8 / (1000 ** 2)} MB")
    print(f"Size/year: {sum(el.nbytes for el in fft) * len(levels) * 8 * 365 / (1000 ** 2)} MB")

In [29]:
fit_dft2_hermite_interpolated_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                                      time=0, quantile=0.99044, skip_levels=4)


  0%|          | 0/36 [00:00<?, ?it/s]

Original Stdev: 10.063274383544922 m/s
Predicted RMSE: 0.7690223521542094 m/s
Predicted MAE:  0.493954062461853 m/s
Size/level: 0.006986 MB
Size/time: 0.06986 MB
Size/day: 0.55888 MB
Size/year: 203.9912 MB
